##**Stochastic Optimization**

## Notebook Setup 
The following cell will install Drake, checkout the manipulation repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  

More details are available [here](http://manipulation.mit.edu/drake.html).

In [1]:
import importlib
import os, sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='fa5bcfb6367cd0cfda0e3d11e11854d68b39478a', drake_version='20201118', drake_build='nightly')

from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# setup ngrok server
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np

import matplotlib
import matplotlib.pyplot as plt, mpld3
if running_as_notebook:
  mpld3.enable_notebook()

import pydrake
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf
from pydrake.all import (Variable, Evaluate)

def loss(theta):
  x = theta[0]
  y = theta[1]
  eval = 2 * x ** 2 - 1.05 * x ** 4 + x ** 6 / 6 + x * y + y ** 2
  return 0.25 * eval

def generate_color_mat(color_vec, shape):
  color_mat = np.tile(np.array(color_vec).astype(np.float32).reshape(3,1), (1, shape[1]))
  return color_mat

def visualize_loss(vis, loss, colormap='viridis', spacing=0.01, clip_min=None, clip_max=None):
  # Create a grid of thetas and evaluate losses.
  points = []
  for i in np.arange(-3, 3, spacing):
    for j in np.arange(-3, 3, spacing):
      points.append([i,j,loss(np.array([i,j]))])
  points = np.array(points).transpose()

  # Normalize losses and color them according to colormap. 
  cmap = matplotlib.cm.get_cmap(colormap)
  colors = []
  min_loss = np.min(points[2,:]) if clip_min == None else clip_min
  max_loss = np.max(points[2,:]) if clip_max == None else clip_max

  for i in range(points.shape[1]):
    normalized_loss = (points[2,i] - min_loss) / (max_loss - min_loss)
    color = list(cmap(normalized_loss))[0:3]
    colors.append(color)

  colors = np.array(colors).transpose()

  vis.delete()
  vis["/Background"].set_property('visible', False)
  vis["/loss"].set_object(g.PointCloud(points, colors, size=0.03))

HEAD is now at fa5bcfb Merge pull request #111 from hjsuh94/rl/sets

+ [[ 0 -ne 0 ]]
+ command -v conda
+ apt-get update -qq
+ apt-get install -o APT::Acquire::Retries=4 -o Dpkg::Use-Pty=0 -qy --no-install-recommends lsb-release
++ lsb_release -cs
+ [[ bionic != \b\i\o\n\i\c ]]
+ apt-get install -o APT::Acquire::Retries=4 -o Dpkg::Use-Pty=0 -qy --no-install-recommends ca-certificates gnupg
+ APT_KEY_DONT_WARN_ON_DANGEROUS_USAGE=1
+ apt-key adv -q --fetch-keys https://bazel.build/bazel-release.pub.gpg
+ echo 'deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8'
+ apt-get update -qq
++ cat
+ apt-get install -o APT::Acquire::Retries=4 -o Dpkg::Use-Pty=0 -qy --no-install-recommends bazel jupyter jupyter-nbconvert jupyter-notebook locales python3 python3-bs4 python3-dev python3-entrypoints python3-future python3-ipywidgets python3-jinja2 python3-jsonschema python3-numpy python3-pandas python3-pip python3-requests python3-retrying python3-setuptools python3-six python3-sn

## The Three Hump Camel 
In this exercise, we'll implement our own versions of gradient descent and stochastic gradient descent! 

Our goal is to find the minima of the following function:

$$l(x)= \frac{1}{4}\bigg(2x_1^2-1.05x_1^4+\frac{x_1^6}{6}+x_1x_2+x_2^2\bigg)$$

Note that you can access this function using `loss(x)`.

We have visualized the landscape of this function in meshcat if you run the cell below! You will notice the following things:

1. This function has 3 local minima (hence, the name 'three hump camel')
2. The global minima is located at $f([0,0])=0$. 

In [2]:
vis = meshcat.Visualizer(zmq_url)
# The paramters are optimized for best visualization in meshcat. 
# For faster visualization, try increasing spacing. 
visualize_loss(vis, loss, colormap = 'viridis', spacing=0.02, clip_max=2.0)

You can open the visualizer by visiting the following URL:
http://172807b55fa6.ngrok.io/static/


## Gradient Descent

As we saw in the lecture, one way of trying to find the minimum of $l(x)$ is to use explicit gradients and do gradient descent. 

$$x \leftarrow x - \eta\bigg(\frac{\partial l(x)}{\partial x}\bigg)^T$$

We've set up a basic outline of the gradient descent algoritm for you. Take a look at the following function `gradient_descent` that implements the following steps:

1. Initialize $x\in\mathbb{R}^2$ at random from some bounded region.
2. Until maximum iteration, update $x$ according to the rule. 

In [4]:
def gradient_descent(rate, update_rule, initial_x=None, iter=1000):
  """gradient descent algorithm 
  @params: 
  - rate (float): eta variable of gradient descent.
  - update_rule: a function with a signature update_rule(x, rate). 
  - initial_x: initial position for gradient descent. 
  - iter: number of iterations to run gradient descent for.
  """
  # If no initial guess is supplied, then randomly choose one. 
  if initial_x is None:
    x = -3 + 6.0 * np.random.rand(2)
  else:
    x = initial_x
  # Compute loss for first parameter for visualization. 
  x_list = []
  x_list.append([x[0], x[1], loss(x)])
  # Loop through with gradient descent. 
  for i in range(iter):
    # Update the parameters using update rule.
    x = update_rule(x, rate)
    x_list.append([x[0], x[1], loss(x)])
  return np.array(x_list)

## Determinisitc Exact Gradients

**Problem 9.1.a** [2 pts]: Let's first use the vanilla gradient descent algorithm with exact gradients. Below, you must implement the simple update function:

$$x \leftarrow x - \eta\bigg(\frac{\partial l(x)}{\partial x}\bigg)^T$$

HINT: You can write down the gradient yourself, but remember you can also use drake's symbolic differentiation!


In [5]:
def exact_gradient(x, rate):
    """
    Update rule. Receive theta and update it with the next theta.
    @params
    - x: input variable x.
    - rate: rate of descent, variable "eta". 
    @returns:
    - updated variable x. 
    """
    x_var = Variable('x')
    y_var = Variable('y')
    L_xy = loss([x_var,y_var])
    J = L_xy.Jacobian([x_var,y_var])
    J_val = np.squeeze(np.array(Evaluate(J,{x_var:x[0], y_var:x[1]})),1)
    x = x - rate*J_val
    return x

When you've completed the function, you can run the below cell to check the visualization! For this problem, the visualization has the following convention:
- Red sphere is the initial guess 
- Green sphere is the final point after `iter` iterations. 
- Every updated parameter is drawn as smaller red cubes. 

In [6]:
# Compute the trajectory. 
trajectory = gradient_descent(0.1, exact_gradient)
vis["/traj"].set_object(g.PointCloud(trajectory.transpose(), generate_color_mat([1, 0, 0], trajectory.transpose().shape), size=0.03))
vis["/traj_line"].set_object(g.Line(g.PointsGeometry(trajectory.transpose()), g.MeshBasicMaterial(color=0xff0000)))

# Visualize the initial guess.
vis["/traj_initial"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0xff0000, reflectivity=0.8))
vis["/traj_initial"].set_transform(tf.translation_matrix(trajectory[0,:]))

# Visualize the final point of the iteration.
vis["/traj_final"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0x00ff00, reflectivity=0.8))
vis["/traj_final"].set_transform(tf.translation_matrix(trajectory[-1,:]))

If you've implemented it correctly, run the cell multiple times to see the behavior of gradient descent from different initial conditions. You should note that depending on where you started, you are deterministically stuck in the local minima that corresponds to its attraction region. 

## Stochastic Approximation to Gradients

**Problem 9.1.b** [2 pts]: One of the mindblowing facts we learned from the lecture was that we can actually do gradient descent without ever having true gradients of the loss function $l(x)$! 

Your job is to write down the following update function for gradient descent:

$$x \leftarrow x - \eta\big[l(x+w)-l(x)\big]w$$

where $w\in\mathbb{R}^2$ drawn from a Gaussian distribution, $w\sim\mathcal{N}(0,\sigma^2=0.25)$. You can use `np.random.normal()` to draw from this distribution.

In [7]:
def approximated_gradient(x, rate):
  """
  Update rule. Receive theta and update it with the next theta.
  @params
  - x: input variable x.
  - rate: rate of descent, variable "eta". 
  @returns:
  - updated varaible x. 
  """
  w = np.random.normal(0,0.5,2)
  x = x - rate * (loss(x+w) - loss(x)) * w
  return x

Again, once you've implemented the function, run the below cell to visualize the trajectory.

In [8]:
trajectory = gradient_descent(0.01, approximated_gradient, iter=10000)
color_mat = np.tile(np.array([1.,0.,0]).astype(np.float32),(30,1)).transpose()
vis["/traj"].set_object(g.PointCloud(trajectory.transpose(), generate_color_mat([1, 0, 0], trajectory.transpose().shape), size=0.03))
vis["/traj_line"].set_object(g.Line(g.PointsGeometry(trajectory.transpose()), g.MeshBasicMaterial(color=0xff0000)))

# Visualize the initial guess.
vis["/traj_initial"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0xff0000, reflectivity=0.8))
vis["/traj_initial"].set_transform(tf.translation_matrix(trajectory[0,:]))

# Visualize the final point of the iteration.
vis["/traj_final"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0x00ff00, reflectivity=0.8))
vis["/traj_final"].set_transform(tf.translation_matrix(trajectory[-1,:]))

If you've implemented it correctly, take a moment to run it from multiple different conditions - the results are somewhat shocking.
- With the right parameters ($\sigma,\eta$), this version of gradient descent is much better than the deterministic exact version at converging to global minima. (In fact, you'll sometimes see it hop out of one of the local minimas and converge to a global minima?)
- But we never explicitly took derivatives!
- (Side note): does this mean this way approximating gradients is the magical tool to everything? not quite. This version can be prone to getting stuck in saddle points!

## Baselines 

**Problem 9.1.c** [4 pts]: We don't necessarily have to take finite differences to estimate the gradient. In fact, we could have subtracted our perturbed estimate from any function, as long as it is not a function of $w$! 

$$x \leftarrow x - \eta\big[l(x+w)-b(x)\big]w$$

As a written problem, the problem is as follows: prove that on average, the difference in the updates (call it $\mathbb{E}_w[\Delta x$]) is approximately equal to the true analytical gradient. 

HINT: Use first-order taylor approximation of $l(x+w)$ (i.e. you may assume $w$ is quite small)

**Problem 9.1.d** [1 pts]: Finally, implement the update law above. The update rule is almost identical to 9.1.b except for the implementation of the baseline, so this is like a bonus question.  

In [9]:
def approximated_gradient_with_baseline(x, rate, baseline):
  """
  Update rule. Receive theta and update it with the next theta.
  @params
  - x: input variable x.
  - rate: rate of descent, variable "eta". 
  - baseline: float for baseline.
  @returns:
  - updated varaible x. 
  """
  w = np.random.normal(0,0.5,2)
  x = x - rate * (loss(x+w) - baseline(x)) * w
  return x

As you proved in 9.1.c, adding a baseline does not change the mean of the update. However, it does change the variance!

In the below code, you can play around with different values of the baseline to see what happens. Remember that the optimal value (smallest variance) of the baseline is $l(x)$. 

You should see that if the baseline is close to `loss(x)` (e.g. baseline is uniformly zero), there is no big difference with the solution you wrote on 9.1.b. However, when the baseline is far from `loss(x)` (e.g. baseline is uniformly 5), our path starts to look more like a random walk due to high variance.

In [10]:
def baseline(theta):
    x = theta[0]
    y = theta[1]
    eval = 2 * x ** 2 + y ** 2
    return 0.25 * eval

def reduced_function(x, rate):
  return approximated_gradient_with_baseline(x, rate, baseline)

trajectory = gradient_descent(0.01, reduced_function, iter=10000)
color_mat = np.tile(np.array([1.,0.,0]).astype(np.float32),(30,1)).transpose()
vis["/traj"].set_object(g.PointCloud(trajectory.transpose(), generate_color_mat([1, 0, 0], trajectory.transpose().shape), size=0.03))
vis["/traj_line"].set_object(g.Line(g.PointsGeometry(trajectory.transpose()), g.MeshBasicMaterial(color=0xff0000)))

vis["/traj_initial"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0x00ff00, reflectivity=0.8))
vis["/traj_initial"].set_transform(tf.translation_matrix(trajectory[0,:]))

vis["/traj_final"].set_object(g.Sphere(0.05), g.MeshLambertMaterial(color=0xff0000, reflectivity=0.8))
vis["/traj_final"].set_transform(tf.translation_matrix(trajectory[-1,:]))

## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

For submission of this assignment, you must do two things. 
- Download and submit the notebook `stochastic_optimization.ipynb` to Gradescope's notebook submission section, along with your notebook for the other problems.
- Write down your answers to 9.1.c to a separately pdf file and submit it to Gradescope's written submission section. 

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [2 pts] 9.1.a must be implemented correctly.
- [2 pts] 9.1.b must be implemented correctly.
- [4 pts] 9.1.c is answered correctly.
- [1 pts] 9.1.d must be implemented correctly.

In [11]:
from manipulation.exercises.rl.test_stochastic_optimization import TestStochasticOptimization
from manipulation.exercises.grader import Grader 

Grader.grade_output([TestStochasticOptimization], [locals()], 'results.json')
Grader.print_test_results('results.json')

Total score is 5/5.

Score for Test approximated gradient function is 2/2.

Score for Test approximated gradient with baseline function is 1/1.

Score for Test exact gradient function is 2/2.
